In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils as utils
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions import Distribution
from torch.distributions.dirichlet import Dirichlet
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
import math
from typing import Tuple, List, Callable

from sklearn.preprocessing import StandardScaler
from torchvision import datasets, transforms
from torchvision import utils as vutils
from sklearn import metrics
from tqdm import tqdm
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from scipy.stats import gaussian_kde
from IPython.display import clear_output

from sklearn.datasets import make_moons

import wandb
import os 
os.environ['WANDB_NOTEBOOK_NAME'] = 'Coupling_mnist_plots.ipynb'
# set seed
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

############ HYPERPARAMETERS ################

#lr = 0.00005
training_lr = 5e-5
start_lr = 1e-9
min_lr = 1e-7 # during cosine annealing
num_epochs = 200 # flere epochs maybe 12000
warmup_steps= 500
validation_every_steps = 50 # is actually every epoch in training loop!!
#validation_every_epochs = 1
weight_decay = 1e-7  # L2 regularization strength to prevent overfitting in Adam or AdamW 
batch_size = 64
early_stop_delta = 0.0001
early_stop_patience = 12
split = lambda x: x.chunk(2, dim=-1)
reg = 5e-5#  entropy regularisation
annealing_interval = 200 # Every 10 epochs, anneal LR (warm restart)

num_classes = 2
latent_dim = 2 # the encoder outputs 2D latent space
data_dim = 2 # the encoder outputs 2D latent space
in_dim= data_dim // 2 # since we split the data
out_dim= data_dim // 2
num_params = 2 # s and t
num_hidden = 2 # number of hidden layers
hidden_dim = 32 # neurons in hidden layers
num_flows = 4 # number of coupling flow layers


wandb.init(
    project='Normalising-Flow-DNN',
    config={
        'architecture': 'PostNet',
        'dataset': 'TwoMoons',
        'training_lr': training_lr,
        'start_lr': start_lr,
        'min_lr': min_lr,
        'num_epochs': num_epochs,
        'warmup_steps': warmup_steps,
        'validation_every_steps': validation_every_steps,
        'weight_decay': weight_decay,
        'batch_size': batch_size,
        'early_stop_delta': early_stop_delta,
        'early_stop_patience': early_stop_patience,
        'reg': reg,
        'annealing_interval': annealing_interval,
        'num_classes': num_classes,
        'latent_dim': latent_dim,
        'num_params': num_params,
        'num_hidden': num_hidden,
        'hidden_dim': hidden_dim,
        'num_flows': num_flows,
    }
    #name='run_name',
    #tags=['experiment1', ''],
    #notes='Trying out a new architecture',
    #dir='/path/to/log/files',
    ##entity='my_team',
    #group='experiment_group',
    #job_type='train'
)

############ CLASSES ################

from postnet.PosteriorNetwork_class import PosteriorNetwork, Conditioner, AffineCouplingLayer, NormalisingFlow
from postnet.Encoder_Moons import Encoder_Moons
from postnet.Encoder_MNIST import Encoder_MNIST
from postnet.Encoder_CIFAR import Encoder_CIFAR

    
############ LOAD MOONS DATASET ################
from torch.utils.data import DataLoader, TensorDataset
# Generate the "make_moons" dataset
X, y = make_moons(n_samples=3000, noise=0.1, random_state=42)

# 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Create PyTorch datasets and data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

# Dictionary with class names
classes = {0: "Class 0", 1: "Class 1"}

# Initialize dictionary to store the counts for each class
N = {class_idx: (y_train == class_idx).sum().item() for class_idx in classes}
N = torch.tensor([N[class_idx] for class_idx in classes])

############# FUNCTION ##################

def init_weights(model):
    if isinstance(model, nn.Linear):
        nn.init.xavier_normal_(model.weight)
        nn.init.zeros_(model.bias) 

def accuracy(y_train, preds):
    accuracy = accuracy_score(y_train.cpu().numpy(), preds.cpu().numpy())
    return accuracy

############# INSTANTIATE MODEL ##################
flow_models = []
for class_label in range(num_classes):
    conditioner = Conditioner(in_dim=in_dim, out_dim=out_dim, num_hidden=num_hidden, hidden_dim=hidden_dim, num_params=num_params)
    affine_coupling = AffineCouplingLayer(conditioner, split=split) # split the tensor into 2 parts
    flows = [affine_coupling for _ in range(num_flows)]
    latent_distribution = torch.distributions.MultivariateNormal(loc=torch.zeros(data_dim).to(device), scale_tril=torch.eye(data_dim).to(device)) #maybe move out of loop?

    flow_model = NormalisingFlow(latent_distribution, flows).to(device)
    #flow_model = NormalisingFlow(latent_distribution, flows).apply(init_weights).to(device)
    flow_models.append(flow_model)

postnet_model = PosteriorNetwork(latent_dim, flow_models, N, num_classes, y_train, reg).to(device) 
optimiser = optim.AdamW(postnet_model.parameters(), lr=training_lr, weight_decay=weight_decay)

class GradualWarmupScheduler(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_steps, start_lr, end_lr, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.lr_increment = (end_lr - start_lr) / warmup_steps
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.warmup_steps:
            lr = self.start_lr + self.last_epoch * self.lr_increment
            return [lr for _ in self.base_lrs]
        return self.base_lrs
warmup_scheduler = GradualWarmupScheduler(optimiser, warmup_steps=warmup_steps, start_lr=start_lr, end_lr=training_lr)

total_steps_per_epoch = len(train_loader)  # Total batches (steps) per epoch
warmup_epochs = math.ceil(warmup_steps / total_steps_per_epoch)  # Total warmup epochs
#T_max = num_epochs - warmup_epochs  # Total number of training epochs minus warmup epochs
print("total_steps_per_epoch", total_steps_per_epoch)
print("Warmup epochs:", warmup_epochs)
#print("Training epochs T_max: ", T_max)
# rounded up to make sure all warmup steps are used before AnnealingLR

#T_max = 10
training_scheduler = lr_scheduler.CosineAnnealingLR(optimiser, T_max=annealing_interval, eta_min=min_lr, last_epoch=-1)
#training_scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=5, gamma=0.1)
#training_scheduler = ExponentialLR(optimizer=default_optimiser, gamma=0.98)

Device: cuda


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fenriswulven (dtu_projects). Use `wandb login --relogin` to force relogin


ModuleNotFoundError: No module named 'postnet'

In [ ]:
def plot_latent_z(model, X_train, y_train):
    latent_train_data = model.cnn(X_train.to(device)).cpu().detach().numpy()
    y_train = y_train.cpu().detach().numpy()
    # Create a buffer to save the plot
    buf = io.BytesIO()
    
    plt.figure(figsize=(6,4))
    plt.scatter(latent_train_data[y_train== 0][:, 0], latent_train_data[y_train== 0][:, 1], c='red', label='True Class 0', marker='o')
    plt.scatter(latent_train_data[y_train== 1][:, 0], latent_train_data[y_train== 1][:, 1], c='green', label='True Class 1', marker='o')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Latent Space')
    plt.legend(loc='upper left')
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = plt.imread(buf, format='png')
    wandb.log({"Latent representations": wandb.Image(image)})
    plt.close()

In [ ]:
############## TRAINING ######################
def train(model, optimiser, train_loader, test_loader, num_epochs, validation_every_steps, 
          early_stop_delta, early_stop_patience, warmup_scheduler, training_scheduler, warmup_steps):
    model.train()
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []
    all_train_losses = []
    best_val_loss = float("Inf")
    step = 0 # how many batches we have trained on (each batch is 64 samples) #9000 training samples / 64 batch size = 140 batches per epoch
    counter = 0 # for early stopping 
    early_stopping = False
    wandb.watch(model, log="all")

    for epoch in range(num_epochs): #epoch is one forward pass through the entire training set
        train_losses_batches, train_accuracies_batches = [], []
        #batches_counter = 0

        for batch_index, (X_train, y_train) in enumerate(train_loader):
            X_train, y_train = X_train.to(device), y_train.to(device)
            # batches_counter += 1

            # Forward pass
            alpha = model(X_train, N)
            loss = model.loss_postnet(alpha, y_train) #batch size
            # Perform one training step
            optimiser.zero_grad()
            loss.backward()
            utils.clip_grad_norm_(flow_model.parameters(), max_norm=5.0)
            
            optimiser.step()
            if step < warmup_steps:
                warmup_scheduler.step()
            
            step += 1
            #train_losses.append(loss.item())

            # Compute training accuracy and loss for this batch
            with torch.no_grad():
                preds = torch.max(alpha, dim=-1)[1]
                train_accuracy_batch = accuracy(y_train, preds)
                train_accuracies_batches.append(train_accuracy_batch)
                train_losses_batches.append(loss.item())
                all_train_losses.append(loss.item())
                current_lr = optimiser.param_groups[0]['lr']
                wandb.log({"batch_train_losses": loss.item(), "batch_train_accuracy": 
                           train_accuracy_batch, "step": step, "learning_rate": current_lr, "epoch": epoch})
                #wandb.log({"batch_train_losses": loss.item(), "batch_train_accuracy": 
                           #train_accuracy_batch, "step": step})
                
                #train_accuracies.append(batch_accuracy)

            # if epoch >= warmup_epochs:
            #     if (epoch - warmup_epochs) % annealing_interval == 0:
            #         training_scheduler.step()

            if step % validation_every_steps == 0:
                train_loss = np.mean(train_losses_batches)
                train_losses.append(train_loss)
                train_accuracy = np.mean(train_accuracies_batches)
                train_accuracies.append(train_accuracy)
                wandb.log({"train_loss": train_loss, "train_accuracy": train_accuracy, "step": step})

                val_losses_batches = []
                #val_accuracies_batches = []
                val_correct = []
                model.eval()
                with torch.no_grad():   
                    for batch_index, (X_test, y_test) in enumerate(test_loader):
                        X_test, y_test = X_test.to(device), y_test.to(device)
                        # Evaluation Forward pass
                        alpha = model(X_test, N) # gives a vector with alphas for each class
                        loss = model.loss_postnet(alpha, y_test) #gives a loss
                        
                        # Evaluation accuracy and loss for this batch
                        preds = torch.max(alpha, dim=-1)[1]
                        
                        correct_batch = (preds == y_test).sum().item() 
                        val_correct.append(correct_batch)

                        #Maybe: Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                        #val_accuracies_batches.append(accuracy(y_test, preds) * len(X_test))

                        # append the loss for this batch
                        val_losses_batches.append(loss.item())

                val_accuracy = sum(val_correct) / len(y_test) #Multiply by len(test_dataset) because the final batch of DataLoader may be smaller (drop_last=False).
                val_accuracies.append(val_accuracy)
                val_loss = np.mean(val_losses_batches) 
                val_losses.append(val_loss)
                wandb.log({"val_loss": val_loss, "val_accuracy": val_accuracy, "step": step})
                model.train()

                if val_losses[-1] < -1.:
                    print("Unstable training")
                    break
                if np.isnan(val_losses[-1]):
                    print('Detected NaN Loss')
                    break
                # If val_loss is the best so far, save the model state_dict and reset the early stopping counter
                if val_losses[-1] < best_val_loss:
                    best_val_loss = val_losses[-1]
                    counter = 0
                    best_model = model.state_dict()
                    torch.save({'epoch': epoch, 'model_state_dict': best_model, 'loss': best_val_loss}, 'best_model_moons_entropy.pth')
                    print('Model saved')
                    # Plot latent space representations z 
                    plot_latent_z(model, X_train, y_train)
                    

                # Early stopping - if val_loss is not improving (plus a delta e-4 as buffer) then start counter
                # after patience of a certain number of validations, then stop training
                elif val_losses[-1] > (best_val_loss + early_stop_delta):
                    counter += 1
                    if counter >= early_stop_patience:
                        #print("Early stopping")
                        early_stopping = True
                        break
                
                print(f"Step: {step}, Epoch: {epoch+1}\tTrain Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}")
                #### Lave plots med meshgrid f-funktion af normalising flow undervejs for at se ændringen
                
        # Update training scheduler (annealing LR)
        if epoch >= warmup_epochs:
                #if (epoch - warmup_epochs) % annealing_interval == 0:
                training_scheduler.step()
        #current_lr = optimiser.param_groups[0]['lr']
        #print(f"Epoch {epoch}: Current Epoch LR = {current_lr}")

        if early_stopping: # if true
            print("Early stopping triggered. Exiting training.")
            break  # Break out of the outer loop
    print("Finished training.")
    return train_losses, val_losses, train_accuracies, val_accuracies, all_train_losses #,model

train_losses, val_losses, train_accuracies, val_accuracies, all_train_losses = train(postnet_model, optimiser, train_loader, test_loader, 
                                                num_epochs, validation_every_steps, early_stop_delta, early_stop_patience, warmup_scheduler, 
                                                training_scheduler, warmup_steps)

In [ ]:
# log figurer op undervejs - på serveren

# efter en fornemmelse af parametrene - tune manuelt først
#- har en gridserach og adaptive tuner

# learning rate
# weight decay - prøve at sætte en hård grænse hvor man ganger med 0.1

# ved 15000 steps: dividere learning rate med 10

#- Plot two moons og bagrund logprobs for Klassen. Grid er datapunkter – køre gennem encoder – og så gennem flow og regne logprobs og plot der. Kan se hvad modellen gør.

#- Plot med outlier og bestemte punkter highlighted. Evt bruge regnbue farver. Husk punkt 5 imellem de to klasser. Out of distribution – aleatorisk usikkerhed. 
  #  Brug AU-ROC curve til at se thresholds for ood data.


In [ ]:
# Plot loss of training and validation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(train_losses, label='Train Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_xlabel('Validation Epochs')
axes[0].set_ylabel('Loss')
axes[0].set_title('Train and Validation Loss')
axes[0].legend()

# Plot accuracies of training and validation
axes[1].plot(train_accuracies, label='Train Accuracy')
axes[1].plot(val_accuracies, label='Validation Accuracy')
axes[1].set_xlabel('Validation Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Train and Validation Accuracy')
axes[1].legend()
plt.tight_layout()
plt.show()

# all train losses
print(len(all_train_losses))
# plot all_train_losses
plt.figure(figsize=(12,8))
plt.plot(all_train_losses,  '.',label='Train Loss', alpha=0.3)
#plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Train Loss')
plt.legend()
plt.show()

In [ ]:
# Load the best model from pth file type
device = torch.device("cpu")
flow_models = []
for class_label in range(num_classes):
    conditioner = Conditioner(in_dim=in_dim, out_dim=out_dim, num_hidden=num_hidden, hidden_dim=hidden_dim, num_params=num_params)
    affine_coupling = AffineCouplingLayer(conditioner, split=lambda x: x.chunk(2, dim=-1)) # split the tensor into 2 parts
    flows = [affine_coupling for _ in range(num_flows)]
    latent_distribution = torch.distributions.MultivariateNormal(loc=torch.zeros(data_dim).to(device), scale_tril=torch.eye(data_dim).to(device))

    flow_model = NormalisingFlow(latent_distribution, flows, ). to(device)
    #flow_model = NormalisingFlow(latent_distribution, flows).apply(init_weights).to(device)
    flow_models.append(flow_model)
best_model = PosteriorNetwork(latent_dim, flow_models, N, num_classes, y_train, reg).to(device)
best_model.load_state_dict(torch.load('best_model_moons_entropy.pth', map_location=device)['model_state_dict'])
#best_model.eval()

# Test best model on test data
N = N.to(device) #move num classes to device just in case
alpha_values = []
# print(next(best_model.parameters()).is_cuda)
best_model.eval()
with torch.no_grad():
    for batch_index, (X_test_batch, y_test_batch) in enumerate(test_loader):
        X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
        # Forward pass to compute alpha values for this batch
        alpha_preds = best_model(X_test_batch, N) # batch x num_classes    
        # Append the alpha values to the list
        #print(alpha_preds.shape)
        alpha_values.append(alpha_preds)

# Combine alpha values from all batches list into a single tensor
alpha_values = torch.cat(alpha_values, dim=0)
print("Alpha values shape: ", alpha_values.shape)

# Normalize alpha values
alphas_norm = F.normalize(alpha_values, p=1, dim=1) #reduce to dim 1, sum to 1

In [ ]:
# plot the latent space
latent_train_data = best_model.cnn(X_train_tensor.to(device)).cpu().detach().numpy()
plt.figure(figsize=(6,4))
plt.scatter(latent_train_data[y_train == 0][:, 0], latent_train_data[y_train == 0][:, 1], c='red', label='True Class 0', marker='o')
plt.scatter(latent_train_data[y_train == 1][:, 0], latent_train_data[y_train == 1][:, 1], c='green', label='True Class 1', marker='o')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Latent Space')
plt.legend(loc='upper left')
plt.show()

#Plot med outlier og bestemte punkter highlighted. Evt bruge regnbue farver. Husk punkt 5 imellem de to klasser. Out of distribution – aleatorisk usikkerhed. 
#  Brug AU-ROC curve til at se thresholds for ood data.
# use X_train and colour them rainbow based on x1 value
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Plot training data
rainbow_colors_c0 = plt.get_cmap('rainbow')(X_train[y_train == 0][:, 0] / X_train[y_train == 0][:, 0].max())
rainbow_colors_c1 = plt.get_cmap('rainbow')(X_train[y_train == 1][:, 0] / X_train[y_train == 1][:, 0].max())

# axes[0].scatter(X_train[y_train == 0][:, 0], X_train[y_train == 0][:, 1], c='red', label='Class 0')
# axes[0].scatter(X_train[y_train == 1][:, 0], X_train[y_train == 1][:, 1], c='green', label='Class 1')
axes[0].scatter(X_train[y_train == 0][:, 0], X_train[y_train == 0][:, 1], c=rainbow_colors_c0, label='Class 0')
axes[0].scatter(X_train[y_train == 1][:, 0], X_train[y_train == 1][:, 1], c=rainbow_colors_c1, label='Class 1')
axes[0].set_xlabel('Input dimension 1')
axes[0].set_ylabel('Input dimension 2')
axes[0].legend(loc='lower left', frameon=True, labelcolor='black')  # Adjust legend properties
axes[0].set_title('Training Data')

# plot the latent space datapoints and see how they changed
latent_train_data = best_model.cnn(X_train_tensor.to(device)).cpu().detach().numpy()
axes[1].scatter(latent_train_data[y_train == 0][:, 0], latent_train_data[y_train == 0][:, 1], c=rainbow_colors_c0, label='True Class 0', marker='o')
axes[1].scatter(latent_train_data[y_train == 1][:, 0], latent_train_data[y_train == 1][:, 1], c=rainbow_colors_c1, label='True Class 1', marker='o')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend(loc='upper left')
axes[1].set_title('Latent Space')
plt.tight_layout()
plt.show()



In [ ]:

# Plot the meshgrid of the learned latent space
# Create a meshgrid in the latent space
coords = 3.8
denseness = 700
nx, ny = (denseness, denseness)
x = np.linspace(-coords, coords, nx)
y = np.linspace(-coords, coords, ny)
xv, yv = np.meshgrid(x, y)
mesh = np.stack((xv.flatten(), yv.flatten()), axis=1)
mesh_tensor = torch.FloatTensor(mesh).to(device)

# Run the train data through only the CNN using the best_model.pt
latent_train_data = best_model.cnn(X_train_tensor.to(device)).cpu().detach().numpy()
# Run a meshgrid of datapoints through encoder (and then through flow) to get logprobs
latent_mesh = best_model.cnn(mesh_tensor.to(device))

num_samples = 1000
samples, posterior_samples, generated_samples = [], [], []
# Compute log-probabilities for each class
logprobs, logprobs_mesh = [], []
for i in range(num_classes):
    log_prob = flow_models[i].log_prob(mesh_tensor).clamp(min=-10, max=10)
    logprobs.append(log_prob.cpu().reshape((nx, ny)).detach().numpy())
    # run latent_mesh through flow to get logprobs
    log_prob_mesh = flow_models[i].log_prob(latent_mesh).clamp(min=-10, max=10)
    logprobs_mesh.append(log_prob_mesh.cpu().reshape((nx, ny)).detach().numpy())

    # Get samples in the latent space
    samples.append(best_model.flow_models[i].sample(num_samples).to(device)) #self.g(self.latent_sample(num_samples))
    # Sample from the posterior distribution using the NormalisingFlow object
    post = best_model.flow_models[i].latent_sample(num_samples).to(device)
    posterior_samples.append(post)    


# Show what the normalising flow has learned
plt.figure(figsize=(6, 4))
plt.imshow(np.exp(logprobs_mesh[0]), extent=[-2, 2, -2, 2], origin='lower', cmap='viridis', alpha=1)
plt.imshow(np.exp(logprobs_mesh[1]), extent=[-2, 2, -2, 2], origin='lower', cmap='viridis', alpha=0.5)
plt.colorbar(label='Probability Density')
plt.xlabel('Input Dimension 1')
plt.ylabel('Input Dimension 2')
plt.title('Before Encoder Combined Probability Density for Both Classes')
plt.show()

#Grid er datapunkter – køre gennem encoder – og så gennem flow og regne logprobs og plot der
# BEFORE encoder plot the logprobs_mesh as heatmaps for each class
colors = ['red', 'green']
plt.figure(figsize=(12,4))
for class_label in range(num_classes):
    plt.subplot(1, num_classes, class_label + 1)
    plt.imshow(np.exp(logprobs_mesh[class_label]), extent=[-coords, coords, -coords, coords], origin='lower', cmap='viridis')
    plt.colorbar()
    plt.scatter(X_train[y_train == class_label][:, 0], X_train[y_train == class_label][:, 1],
                c=colors[class_label], alpha=0.07, label=f'True Class {class_label}')
    plt.xlabel('Input Dimension 1')
    plt.ylabel('Input Dimension 2')
    plt.legend(loc='upper left')
    plt.title(f'Before encoder Class {class_label} Probability Density map with Train Data ') 
plt.tight_layout()
plt.show()

# AFTER encoder Plot the log probabilities as heatmaps for each class
colors = ['red', 'red']
plt.figure(figsize=(12,4))
for class_label in range(num_classes):
    plt.subplot(1, num_classes, class_label + 1)
    plt.imshow(np.exp(logprobs[class_label]), extent=[-coords, coords, -coords, coords], origin='lower', cmap='viridis')
    plt.colorbar()
    plt.scatter(latent_train_data[y_train == class_label][:, 0], latent_train_data[y_train == class_label][:, 1],
                c=colors[class_label], alpha=0.07, label=f'True Class {class_label}')
    plt.xlabel('Latent Dimension 1')
    plt.ylabel('Latent Dimension 2')
    plt.legend(loc='upper left')
    plt.title(f'After encoder Class {class_label} Probability Density with Train Data')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,4))
for class_label in range(num_classes):
    plt.subplot(1, num_classes, class_label + 1)
    plt.imshow(np.exp(logprobs[class_label]), extent=[-coords, coords, -coords, coords], origin='lower', cmap='viridis')
    plt.colorbar()
    plt.scatter(samples[class_label][:, 0].numpy(), samples[class_label][:, 1].numpy(), 
                c=colors[class_label],s=10, alpha=0.07, label=f'Generated Class {class_label}')
    plt.xlabel('Latent Dimension 1')
    plt.ylabel('Latent Dimension 2')
    plt.legend(loc='upper left')
    plt.title(f'Class {class_label} Probability Density with Generated \nSamples from the normalising flow')
plt.tight_layout()
plt.show()

# Plot the log probabilities for both classes in the same plot
plt.figure(figsize=(8, 6))
plt.imshow(np.exp(logprobs[0]), extent=[-2, 2, -2, 2], origin='lower', cmap='viridis', alpha=1)
plt.imshow(np.exp(logprobs[1]), extent=[-2, 2, -2, 2], origin='lower', cmap='viridis', alpha=0.5)
plt.colorbar(label='Probability Density')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('Combined Probability Density for Both Classes')
plt.show()

In [ ]:
# Plot the meshgrid of the learned latent space
# Create a meshgrid in the latent space
nx, ny = (500, 500)
coords = 2
x = np.linspace(-coords, coords, nx)
y = np.linspace(-coords, coords, ny)
xv, yv = np.meshgrid(x, y)
mesh = np.stack((xv.flatten(), yv.flatten()), axis=1)
mesh_tensor = torch.FloatTensor(mesh).to(device)

# Run the train data through only the CNN using the best_model.pt
latent_train_data = best_model.cnn(X_train_tensor.to(device)).cpu().detach().numpy()
num_samples = 1000
logprobs, generated_samples, posterior_samples = [], [], []
# Compute log-probabilities for each class
logprobs = []
for i in range(num_classes):
    log_prob = flow_models[i].log_prob(mesh_tensor).clamp(min=-10, max=10)
    logprobs.append(log_prob.cpu().reshape((nx, ny)).detach().numpy())

    # Sample from the posterior distribution using the NormalisingFlow object
    posterior_samples.append(best_model.flow_models[i].latent_sample(num_samples).to(device))

    # Run function g on a latent sample to map it to data space
    generated_samples.append(best_model.flow_models[i].sample(num_samples).to(device))

# Plot the log probabilities as heatmaps for each class
colors = ['red', 'green']
plt.figure(figsize=(10, 4))
for class_label in range(num_classes):
    plt.subplot(1, num_classes, class_label + 1)
    plt.imshow(np.exp(logprobs[class_label]), extent=[-coords, coords, -coords, coords], origin='lower', cmap='viridis')
    plt.scatter(latent_train_data[y_train == class_label][:, 0], latent_train_data[y_train == class_label][:, 1],
                c=colors[class_label], label='True Class {}'.format(class_label), marker='.', alpha=0.02)
    plt.legend(loc='upper left')
    plt.colorbar()
    plt.xlabel('Latent Dimension 1')
    plt.ylabel('Latent Dimension 2')
    plt.title(f'Class {class_label} Probability Density')
plt.tight_layout()
plt.show()

# Create subplots for each class
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for i, ax_row in enumerate(axes):
    ax_row[0].scatter(generated_samples[i][:, 0].numpy(), generated_samples[i][:, 1].numpy(), s=10, alpha=0.5)
    ax_row[0].set_xlabel('Feature 1')
    ax_row[0].set_ylabel('Feature 2')
    ax_row[0].set_title(f'Class {i} Generated Samples')

    ax_row[1].scatter(posterior_samples[i][:, 0].numpy(), posterior_samples[i][:, 1].numpy(), s=10, alpha=0.5)
    ax_row[1].set_xlabel('Feature 0')
    ax_row[1].set_ylabel('Feature 0')
    ax_row[1].set_title(f'Class {i} Posterior Samples')

plt.tight_layout()
plt.show()

In [ ]:
# Plot histograms of the alphas and normalized alphas
print(alphas_norm.shape)
fig, ax = plt.subplots(2, 2, figsize=(10, 6))
ax[0,0].hist(alpha_values[y_test == 0][:,0], bins=50, label='Alpha 0', alpha=1)
#ax[0,0].hist(alpha_values[y_test == 0][:,1], bins=50, label='Alpha 1', alpha=0.5, color='red')
ax[0,0].set_title('Histogram of Alphas values for Class 0')
ax[0,0].set_xlabel('Alpha')
ax[0,0].set_ylabel('Frequency')
ax[1,0].hist(alpha_values[y_test == 1][:,1], bins=50)
ax[1,0].set_title('Histogram of Alphas values for Class 1')
ax[1,0].set_xlabel('Alpha')
ax[1,0].set_ylabel('Frequency')
ax[0,1].hist(alphas_norm[y_test == 0][:,0], bins=50)
ax[0,1].set_title('Histogram of Normalized Alphas values for Class 0')
ax[0,1].set_xlabel('Alpha')
ax[0,1].set_ylabel('Frequency')
ax[1,1].hist(alphas_norm[y_test == 1][:,1], bins=50)
ax[1,1].set_title('Histogram of Normalized Alphas values for Class 1')
ax[1,1].set_xlabel('Alpha')
ax[1,1].set_ylabel('Frequency')
plt.tight_layout()
plt.show()